In [ ]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, FewShotPromptTemplate, FewShotChatMessagePromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationBufferWindowMemory(
    k=10,
    memory_key="chat_history",
    return_messages=True,
    )

examples = [
    {
        "input": "Top Gun",
        "output": "🛩️👨‍✈️🔥",
    },
    {
        "input": "The Godfather",
        "output": "👨‍👨‍👦🔫🍝",
    },
    {
        "input": "Frozen",
        "output": "❄️👭✨",
    },
]

example_prompt = ChatPromptTemplate.from_messages([
        ("human", "{input}"),
        ("ai", "{output}")
    ])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    input_variables=["input"]
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You must summarize the movie in 3 words, and then convert each word into an emoji."),
    few_shot_prompt,
    MessagesPlaceholder(variable_name="chat_history"),
    ("human","{input}")
])

def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | final_prompt | llm

def invoke_chain(input):
    result = chain.invoke({"input": input})
    memory.save_context(
        {"input": input},
        {"output": result.content},
    )
    print(result)


In [28]:
invoke_chain("Jurassic World")
invoke_chain("Titanic")

content='🦖🌎🔥'
content='🚢❤️❄️'


In [29]:
invoke_chain("What was the first movie I asked about")

content='🛩️👨\u200d✈️🔥'
